In [ ]:
import pandas as pd

# 섭취기준 데이터 읽기
standard = pd.read_excel("./섭취기준_데이터/(17개 영양소)한국인 영양소 섭취기준.xlsx")
standard

### 폴더에 있는 파일들 for loop로 돌리기 _ listdir 모듈을 사용

설명 링크: https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

링크 내용

os.listdir() will get you everything that's in a directory - files and directories.

If you want just files, you could either filter this down using os.path:

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
or you could use os.walk() which will yield two lists for each directory it visits - splitting into files and dirs for you. If you only want the top directory you can just break the first time it yields

from os import walk

f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break

In [ ]:
# 폴더 내의 모든 파일들을 읽어오는 코드
from os import listdir

onlyfiles = [f for f in listdir("./아침식단_데이터")]
onlyfiles

In [ ]:
import re
import numpy as np
# 전체 파일을 돌면서 영양소 정보를 더해준다.

for file in onlyfiles:
    # 엑셀파일 확장자 삭제
    del_extension = re.sub("[.]xlsx","",file)
    
    # 각 파일의 정보 단위로 쪼갠 리스트 만들기
    # 3번항목: 밥 / 4번항목: 국 / 5번항목: 메인 / 6번항목: 서브
    info_list = del_extension.split("_")
    print(info_list[3],info_list[4],info_list[5],info_list[6])
    # 식단파일을 dataframe으로 읽기
    diet = pd.read_excel("./아침식단_데이터/"+file)
    # column 별로 파일을 읽어서 각 식단의 음식의 영양소 정보를 해당 row에 축적해서 더해준다.
    밥 = pd.read_excel("./음식_데이터/"+info_list[3]+".xlsx")
    국 = pd.read_excel("./음식_데이터/"+info_list[4]+".xlsx")
    메인 = pd.read_excel("./음식_데이터/"+info_list[5]+".xlsx")
    서브 = pd.read_excel("./음식_데이터/"+info_list[6]+".xlsx")
    search_list = ["밥","국","메인","서브"]
    영양소정보_리스트 = {}
    영양소정보_리스트["밥"] = []
    영양소정보_리스트["국"] = []
    영양소정보_리스트["메인"] = []
    영양소정보_리스트["서브"] = []
    for keyword in search_list:
        if keyword == "밥":
            비교파일 = 밥
        elif keyword == "국":
            비교파일 = 국
        elif keyword == "메인":
            비교파일 = 메인
        elif keyword == "서브":
            비교파일 = 서브
        for dish in diet[keyword]:
            음식정보 = 비교파일[비교파일.iloc[:,1]==dish]
            영양소정보 = 음식정보.iloc[:,5:]
            영양소정보_리스트[keyword].append(list(영양소정보.values)[0])
            
    영양소_총계 = np.array(영양소정보_리스트["밥"])+np.array(영양소정보_리스트["국"])+np.array(영양소정보_리스트["메인"])+np.array(영양소정보_리스트["서브"])
    최종데이터 = pd.concat([diet,pd.DataFrame(영양소_총계)],axis = 1)
    최종데이터.columns = ["식단번호"] + list(diet.columns[1:])+list(밥.columns[5:])
    최종데이터.to_excel("(영양소추가)"+file)
            
    
    

In [ ]:
from os import listdir

onlyfiles = [f for f in listdir("./영양소추가_식단추출_데이터")]
onlyfiles

In [ ]:
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
fm._rebuild() # 시스템에 폰트파일 복사 후 최초 1회 수행
# 폰트 설정
fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/NanumPen.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
print('font_name:', font_name)
matplotlib.rc('font', family=font_name)
plt.rcParams.update({'font.size': 40})

fig = plt.figure()
plt.plot([1,1],[3,4])
plt.title("한글")
plt.show() 
%matplotlib inline

In [ ]:
import re
import matplotlib.pyplot as plt

for file in onlyfiles:
    # 엑셀파일 확장자 삭제
    del_extension = re.sub("[.]xlsx","",file)
    del_bracket = re.sub("[(]영양소추가[)]","",del_extension)
    info_list = del_bracket.split("_")
    영양소기준정보 = standard[standard.loc[:,"성별"] == info_list[0]][standard.loc[:,"연령"] == info_list[1]][standard.loc[:,"조합 코드"] == info_list[2]]
    # title로 사용가능: 영양소기준정보["성별"],[연령]
    diet = pd.read_excel("./영양소추가_식단추출_데이터/"+file)
    유사도 = list(diet["유사도"])
    에너지정보 = list(diet["에너지(kcal)"])
    단백질정보 = list(diet["단백질(g)"])
    탄수화물정보 = list(diet["탄수화물(g)"])
    지방정보 = list(diet["지질(g)"])
    당정보 = list(diet["총 당류(g)"])
    나트륨정보 = list(diet["나트륨(mg)"])
    fig = plt.figure(figsize=(20,20))
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)
    ax1 = fig.add_subplot(3, 2, 1)
    ax2 = fig.add_subplot(3, 2, 2)
    ax3 = fig.add_subplot(3, 2, 3)
    ax4 = fig.add_subplot(3, 2, 4)
    ax5 = fig.add_subplot(3, 2, 5)
    ax6 = fig.add_subplot(3, 2, 6)
    # 에너지 비교 그래프
    ax1.set_title("에너지(kcal) 그래프", size=30)
    ax1.vlines(x=영양소기준정보["에너지(kcal/일)"], ymin=0, ymax=max(유사도), color='red')
    ax1.scatter(에너지정보,유사도, s=10, color='blue')
    ax1.set_xlabel("에너지(kcal)",fontsize = 20)
    ax1.set_ylabel("코사인 유사도",fontsize = 20)
    # 단백질 비교 그래프
    ax2.set_title("단백질(g) 그래프", size=30)
    ax2.vlines(x=영양소기준정보["단백질(g/일)"], ymin=0, ymax=max(유사도), color='red')
    ax2.scatter(단백질정보,유사도, s=10, color='blue')
    ax2.set_xlabel("단백질(g/일)",fontsize = 20)
    ax2.set_ylabel("코사인 유사도",fontsize = 20)
    # 탄수화물 비교 그래프
    ax3.set_title("탄수화물(g) 그래프", size=30)
    ax3.vlines(x=영양소기준정보["탄수화물(g/일)"], ymin=0, ymax=max(유사도), color='red')
    ax3.scatter(탄수화물정보,유사도, s=10, color='blue')
    ax3.set_xlabel("에너지(kcal)",fontsize = 20)
    ax3.set_ylabel("코사인 유사도",fontsize = 20)
    # 지방 비교 그래프
    ax4.set_title("지방(g) 그래프", size=30)
    ax4.vlines(x=영양소기준정보["지방(g/일)"], ymin=0, ymax=max(유사도), color='red')
    ax4.scatter(지방정보,유사도, s=10, color='blue')
    ax4.set_xlabel("지방(g)",fontsize = 20)
    ax4.set_ylabel("코사인 유사도",fontsize = 20)
    # 나트륨 비교 그래프
    ax5.set_title("나트륨(mg) 그래프", size=30)
    ax5.vlines(x=영양소기준정보["나트륨(mg/일)"], ymin=0, ymax=max(유사도), color='red')
    ax5.scatter(나트륨정보,유사도, s=10, color='blue')
    ax5.set_xlabel("나트륨(mg)",fontsize = 20)
    ax5.set_ylabel("코사인 유사도",fontsize = 20)
    # 당 비교 그래프
    ax6.set_title("총 당류(g) 그래프", size=30)
    ax6.vlines(x=영양소기준정보["총 당류(g/일)"], ymin=0, ymax=max(유사도), color='red')
    ax6.scatter(당정보,유사도, s=10, color='blue')
    ax6.set_xlabel("총 당류(g)",fontsize = 20)
    ax6.set_ylabel("코사인 유사도",fontsize = 20)
    suptitle = (f"성별: {영양소기준정보['성별'].values[0]} 연령: {영양소기준정보['연령'].values[0]}  조합코드: {영양소기준정보['조합 코드'].values[0]}의 그래프")
    fig.suptitle(suptitle, fontsize = 30)
    plt.savefig(del_bracket+'.png', dpi=1200)            

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np

for file in onlyfiles:
    # 엑셀파일 확장자 삭제
    del_extension = re.sub("[.]xlsx","",file)
    del_bracket = re.sub("[(]영양소추가[)]","",del_extension)
    info_list = del_bracket.split("_")
    영양소기준정보 = standard[standard.loc[:,"성별"] == info_list[0]][standard.loc[:,"연령"] == info_list[1]][standard.loc[:,"조합 코드"] == info_list[2]]
    # title로 사용가능: 영양소기준정보["성별"],[연령]
    diet = pd.read_excel("./영양소추가_식단추출_데이터/"+file)
    diet = diet.iloc[0:10,:]
    # 유사도 = list(diet["유사도"])
    # label = [i for i in range(1,11)]
    colors = np.arange(1,11)
    유사도 = list(diet["유사도"])
    에너지정보 = list(diet["에너지(kcal)"])
    단백질정보 = list(diet["단백질(g)"])
    탄수화물정보 = list(diet["탄수화물(g)"])
    지방정보 = list(diet["지질(g)"])
    당정보 = list(diet["총 당류(g)"])
    나트륨정보 = list(diet["나트륨(mg)"])
    fig = plt.figure(figsize=(20,20))
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)
    ax1 = fig.add_subplot(3, 2, 1)
    ax2 = fig.add_subplot(3, 2, 2)
    ax3 = fig.add_subplot(3, 2, 3)
    ax4 = fig.add_subplot(3, 2, 4)
    ax5 = fig.add_subplot(3, 2, 5)
    ax6 = fig.add_subplot(3, 2, 6)
    # 에너지 비교 그래프
    ax1.set_title("에너지(kcal) 그래프", size=30)
    ax1.vlines(x=영양소기준정보["에너지(kcal/일)"], ymin=0, ymax=max(유사도), color='red',label ="기준선")
    scatter1 = ax1.scatter(에너지정보,유사도, s=10, c=colors)
    ax1.set_xlabel("에너지(kcal)",fontsize = 20)
    ax1.set_ylabel("코사인 유사도",fontsize = 20)
    # produce a legend with the unique colors from the scatter
    legend1 = ax1.legend(*scatter1.legend_elements(),
                        loc="lower left", title="Classes")
    ax1.add_artist(legend1)
    ax1.grid(True)
    # 단백질 비교 그래프
    ax2.set_title("단백질(g) 그래프", size=30)
    ax2.vlines(x=영양소기준정보["단백질(g/일)"], ymin=0, ymax=max(유사도), color='red')
    scatter2 = ax2.scatter(단백질정보,유사도, s=10, c=colors)
    ax2.set_xlabel("단백질(g/일)",fontsize = 20)
    ax2.set_ylabel("코사인 유사도",fontsize = 20)
    legend2 = ax2.legend(*scatter2.legend_elements(),
                        loc="lower left", title="Classes")
    ax2.add_artist(legend2)
    ax2.grid(True)
    # 탄수화물 비교 그래프
    ax3.set_title("탄수화물(g) 그래프", size=30)
    ax3.vlines(x=영양소기준정보["탄수화물(g/일)"], ymin=0, ymax=max(유사도), color='red')
    scatter3 = ax3.scatter(탄수화물정보,유사도, s=10, c=colors)
    ax3.set_xlabel("에너지(kcal)",fontsize = 20)
    ax3.set_ylabel("코사인 유사도",fontsize = 20)
    legend3 = ax3.legend(*scatter3.legend_elements(),
                        loc="lower left", title="Classes")
    ax3.add_artist(legend3)
    ax3.grid(True)
    # 지방 비교 그래프
    ax4.set_title("지방(g) 그래프", size=30)
    ax4.vlines(x=영양소기준정보["지방(g/일)"], ymin=0, ymax=max(유사도), color='red')
    scatter4 = ax4.scatter(지방정보,유사도, s=10, c=colors)
    ax4.set_xlabel("지방(g)",fontsize = 20)
    ax4.set_ylabel("코사인 유사도",fontsize = 20)
    legend4 = ax4.legend(*scatter4.legend_elements(),
                        loc="lower left", title="Classes")
    ax4.add_artist(legend4)
    ax4.grid(True)
    # 나트륨 비교 그래프
    ax5.set_title("나트륨(mg) 그래프", size=30)
    ax5.vlines(x=영양소기준정보["나트륨(mg/일)"], ymin=0, ymax=max(유사도), color='red')
    scatter5 = ax5.scatter(나트륨정보,유사도, s=10, c=colors)
    ax5.set_xlabel("나트륨(mg)",fontsize = 20)
    ax5.set_ylabel("코사인 유사도",fontsize = 20)
    legend5 = ax5.legend(*scatter5.legend_elements(),
                        loc="lower left", title="Classes")
    ax5.add_artist(legend5)
    ax5.grid(True)
    # 당 비교 그래프
    ax6.set_title("총 당류(g) 그래프", size=30)
    ax6.vlines(x=영양소기준정보["총 당류(g/일)"], ymin=0, ymax=max(유사도), color='red')
    scatter6 = ax6.scatter(당정보,유사도, s=10, c=colors)
    ax6.set_xlabel("총 당류(g)",fontsize = 20)
    ax6.set_ylabel("코사인 유사도",fontsize = 20)
    legend6 = ax6.legend(*scatter6.legend_elements(),
                        loc="lower left", title="Classes")
    ax6.add_artist(legend6)
    ax6.grid(True)
    suptitle = (f"성별: {영양소기준정보['성별'].values[0]} 연령: {영양소기준정보['연령'].values[0]}  조합코드: {영양소기준정보['조합 코드'].values[0]}의 그래프")
    fig.suptitle(" [상위 10개 데이터 그래프]  "+suptitle, fontsize = 30)
    plt.savefig(" (상위10개) "+del_bracket+'.png', dpi=1200) 
    break

## 참고 데이터 

In [ ]:
import matplotlib.pyplot as plt

horiz_line = 0.0005
vert_line = 110
x_data = [10, 30, 50, 70, 90, 110, 130, 150, 170, 190, 210, 230, 250, 270, 290, 310, 330, 350, 370, 390, 410, 430, 450, 470, 490]
y_data = [0.0074999999999999997, 0.011875, 0.0057812499999999999, 0.0036458333333333334, 0.0020312500000000001, 0.0013125000000000001, 0.00098958333333333342, 0.00089285714285714283, 0.00074218750000000001, 0.00093749999999999997, 0.00071874999999999999, 0.00088068181818181821, 0.00078125000000000004, 0.0004807692307692308, 0.00055803571428571425, 0.00083333333333333339, 0.00066406250000000005, 0.00069852941176470592, 0.00059027777777777778, 0.00059210526315789478, 0.00062500000000000001, 0.0007291666666666667, 0.00068181818181818187, 0.00059782608695652171, 0.00053385416666666663]

fig = plt.figure() # create the top-level container

# horizontal line
plt.hlines(y=horiz_line, xmin=0, xmax=max(x_data), color='red', zorder=1)

# vertical line
plt.vlines(x=vert_line, ymin=0, ymax=max(y_data), color='red', zorder=2)

plt.scatter(x_data, y_data, s=150, color='blue', zorder=3)

plt.show()

In [ ]:
import numpy as np
np.random.seed(19680801)
import matplotlib.pyplot as plt


fig, ax = plt.subplots()
for color in ['tab:blue', 'tab:orange', 'tab:green']:
    n = 750
    x, y = np.random.rand(2, n)
    scale = 200.0 * np.random.rand(n)
    ax.scatter(x, y, c=color, s=scale, label=color,
               alpha=0.3, edgecolors='none')

ax.legend()
ax.grid(True)

plt.show()

In [ ]:
N = 45
x, y = np.random.rand(2, N)
c =  np.random.normal(size=10)
s = np.random.randint(10, 220, size=N)

fig, ax1 = plt.subplots()

ax1.set_title("에너지(kcal) 그래프", size=30)
ax1.vlines(x=영양소기준정보["에너지(kcal/일)"], ymin=0, ymax=max(유사도), color='red',label ="기준선")
scatter = ax1.scatter(에너지정보,유사도, s=10, c=colors)
ax1.set_xlabel("에너지(kcal)",fontsize = 20)
ax1.set_ylabel("코사인 유사도",fontsize = 20)
# produce a legend with the unique colors from the scatter
legend1 = ax1.legend(*scatter.legend_elements(),
                    loc="lower left", title="Classes")
ax1.add_artist(legend1)
ax1.grid(True)

plt.show()

In [ ]:
import numpy as np
np.random.seed(19680801)
import matplotlib.pyplot as plt


N = 45
x, y = np.random.rand(2, N)
c = np.random.randint(1, 5, size=N)
s = np.random.randint(10, 220, size=N)

fig, ax = plt.subplots()

scatter = ax.scatter(x, y, c=c, s=s)

print(scatter.legend_elements())

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower left", title="Classes")
ax.add_artist(legend1)

# produce a legend with a cross section of sizes from the scatter
handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
legend2 = ax.legend(handles, labels, loc="upper right", title="Sizes")

plt.savefig("scatter_legend.png" ,dip = 300)
plt.show()

In [ ]:
import re

# 전체 파일을 돌면서 영양소 정보를 더해준다.

for file in onlyfiles:
    # 엑셀파일 확장자 삭제
    del_extension = re.sub("[.]xlsx","",onlyfiles[0])
    
    # 각 파일의 정보 단위로 쪼갠 리스트 만들기
    # 3번항목: 밥 / 4번항목: 국 / 5번항목: 메인 / 6번항목: 서브
    info_list = del_extension.split("_")
    
    # 식단파일을 dataframe으로 읽기
    diet = pd.read_excel("./식단추출_데이터/"+file)
    
    # column 별로 파일을 읽어서 각 식단의 음식의 영양소 정보를 해당 row에 축적해서 더해준다.
    밥 = pd.read_excel("./음식_데이터/"+info_list[3]+".xlsx")
    국 = pd.read_excel("./음식_데이터/"+info_list[4]+".xlsx")
    메인 = pd.read_excel("./음식_데이터/"+info_list[5]+".xlsx")
    서브 = pd.read_excel("./음식_데이터/"+info_list[6]+".xlsx")
    밥데이터 = 밥[밥.iloc[:,0]=="쌀밥"]
    밥데이터
    break

In [ ]:
밥데이터.iloc[:,5:]

In [ ]:
list_1 = [[1,2,3,4],["a","b","c","d"]]

sample_df = pd.DataFrame(list_1)

print(sample_df.iloc[0].T)

In [ ]:
pd.concat([sample_df,밥데이터.iloc[:,5:]],axis = 1)

In [ ]:
sample_df.join(밥데이터.iloc[:,5:], how='outer')

In [ ]:
listkk = [[1,2,3,4],[1,2,3,4]]
listqq = [[3,4,5,6],[3,4,5,6]]

# pd.DataFrame(np.array([listkk,listqq]),columns = ["영양소1","영양소2","영양소3","영양소4"])

In [ ]:
np.array(listkk)+np.array(listqq)

In [ ]:
최종데이터